# <center>RumbleDB sandbox</center>


This is a RumbleDB notebook that allows you to play with simple JSONiq queries.

It is a jupyter notebook that you can also download and execute on your own machine, but if you arrived here from the RumbleDB website, it is likely to be shown within Google's Colab environment.

To get started, we first need to make sure Java 17 (or 21) is installed.

The following cell was prepared to install Java when using Google Colab (or for Ubuntu generally), which is the setup that jsoniq.org and rumbledb.org link to. If you downloaded the notebook and have another Operating System than Linux, you need to use the Java install command corresponding to your Operating System for this to work.

The warnings can be ignored.

In [ ]:
# We make sure Java 17 is installed.
!apt update
!apt install openjdk-17-jdk


Now we check the Java version. It should return 17. If it does not say 17 or 21, then this needs to be fixed for the rest of the notebook to work.

In [ ]:
!java -version

Now we install the jsoniq Python library.

In [ ]:
%pip install jsoniq

Here, the JSONiq queries are executed locally to the notebook. Advanced users can also run a large Spark cluster and execute JSONiq queries on Petabytes of data.

JSONiq queries can generally be evaluated in Python as follows. It is possible to provide as input Python dicts, lists, pandas DataFrames, and it is possible to retrieve the results as Python values, pandas DataFrames, etc.

In [1]:
from jsoniq import RumbleSession

rumble = RumbleSession.builder.getOrCreate();

print(rumble.jsoniq('{ "foo": [ 6*7 ] }').json());

25/08/29 11:53:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


({'foo': [42]},)


However, for convenience, we activate the jsoniq magic. This means you can directly type the JSONiq queries in a notebook cell, without needing to wrap them inside rumble.jsoniq() calls. This makes it easier to read.

In [2]:
%load_ext jsoniqmagic
# We will display the first 5 items. This parameter can be changed.
rumble.getRumbleConf().setResultSizeCap(5)
rumble.getRumbleConf().setShowErrorInfo(True)


In [3]:
%%jsoniq -df
for $i in 1 to 100000 return {"foobar":1}

DataFrame requested.
+------+
|foobar|
+------+
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
+------+
only showing top 20 rows


## JSON

As explained on the [official JSON Web site](http://www.json.org/), JSON is a lightweight data-interchange format designed for humans as well as for computers. It supports as values:
- objects (string-to-value maps)
- arrays (ordered sequences of values)
- strings
- numbers
- booleans (true, false)
- null

JSONiq provides declarative querying and updating capabilities on JSON data.

## Elevator Pitch

JSONiq is based on XQuery, which is a W3C standard (like XML and HTML). XQuery is a very powerful declarative language that originally manipulates XML data, but it turns out that it is also a very good fit for manipulating JSON natively.
JSONiq, since it extends XQuery, is a very powerful general-purpose declarative programming language. Our experience is that, for the same task, you will probably write about 80% less code compared to imperative languages like JavaScript, Python or Ruby. Additionally, you get the benefits of strong type checking without actually having to write type declarations.
Here is an appetizer before we start the tutorial from scratch.


In [4]:
%%jsoniq -df

let $stores :=
[
  { "store number" : 1, "state" : "MA" },
  { "store number" : 2, "state" : "MA" },
  { "store number" : 3, "state" : "CA" },
  { "store number" : 4, "state" : "CA" }
]
let $sales := [
   { "product" : "broiler", "store number" : 1, "quantity" : 20  },
   { "product" : "toaster", "store number" : 2, "quantity" : 100 },
   { "product" : "toaster", "store number" : 2, "quantity" : 50 },
   { "product" : "toaster", "store number" : 3, "quantity" : 50 },
   { "product" : "blender", "store number" : 3, "quantity" : 100 },
   { "product" : "blender", "store number" : 3, "quantity" : 150 },
   { "product" : "socks", "store number" : 1, "quantity" : 500 },
   { "product" : "socks", "store number" : 2, "quantity" : 10 },
   { "product" : "shirt", "store number" : 3, "quantity" : 10 }
]
let $join :=
  for $store in $stores[], $sale in $sales[]
  where $store."store number" = $sale."store number"
  return {
    "nb" : $store."store number",
    "state" : $store.state,
    "sold" : $sale.product
  }
return [$join]



DataFrame requested.
+--------------------+
|             __value|
+--------------------+
|[{1, MA, broiler}...|
+--------------------+



## All JSON values are JSONiq, too

The first thing you need to know is that a well-formed JSON document is a JSONiq expression as well.
This means that you can copy-and-paste any JSON document into a query. The following are JSONiq queries that are "idempotent" (they just output themselves):

In [5]:
%%jsoniq
{ "pi" : 3.14, "sq2" : 1.4 }

{
  "pi": 3.14,
  "sq2": 1.4
}


In [6]:
%%jsoniq
[ 2, 3, 5, 7, 11, 13 ]

[
  2,
  3,
  5,
  7,
  11,
  13
]


In [7]:
%%jsoniq
{
      "operations" : [
        { "binary" : [ "and", "or"] },
        { "unary" : ["not"] }
      ],
      "bits" : [
        0, 1
      ]
    }

{
  "operations": [
    {
      "binary": [
        "and",
        "or"
      ]
    },
    {
      "unary": [
        "not"
      ]
    }
  ],
  "bits": [
    0,
    1
  ]
}


In [8]:
%%jsoniq
[ { "Question" : "Ultimate" }, ["Life", "the universe", "and everything"] ]

[
  {
    "Question": "Ultimate"
  },
  [
    "Life",
    "the universe",
    "and everything"
  ]
]


This works with objects, arrays (even nested), strings, numbers, booleans, null.

It also works the other way round: if your query outputs an object, you can use it as a JSON document.
JSONiq is a declarative language. This means that you only need to say what you want - the compiler will take care of the how. 

In the above queries, you are basically saying: I want to output this JSON content, and here it is.

In fact JSONiq makes JSON "dynamic": try to replace numbers with arithmetic formulas, keys with concatenations of strings, etc and see how the resulting JSON object is automatically created.

In [9]:
%%jsoniq
{
    "foo" : 2 + 2,
    "foo" || "bar" : if(2 gt 1) then true else false
}

{
  "foo": 4,
  "foobar": true
}


## Navigating an existing JSON dataset

Next, let us look at an existing dataset on the Web. We picked a [GitHub archive file](https://gharchive.org)
that we stored for convenience at this location: https://www.rumbledb.org/samples/git-archive.json.

Accessing a JSON dataset can be done in two ways depending on the exact format:

- If this is a file that contains a single JSON object spread over multiple lines, use json-doc(URL).
- If this is a file that contains one JSON object per line (JSON Lines), use json-lines(URL).

The GitHub archive dataset is in the JSON Lines format, so we open it with json-lines.

In [11]:
%%jsoniq -pdf
json-lines("http://www.rumbledb.org/samples/git-archive-small.json")

,actor,created_at,id,org,payload,public,repo,type
0,(https://avatars.githubusercontent.com/u/20090...,2018-01-01T15:00:00Z,7045118886,None,"(None, d6fce97b8de28a31d021c9a9f7ac939baa14d20...",True,"(115387592, lainrose/Python-Grammar, https://a...",PushEvent
1,(https://avatars.githubusercontent.com/u/17426...,2018-01-01T15:00:00Z,7045118891,None,"(None, 4988f2985f79262054178aa49c3f20b71aca5e8...",True,"(115922595, tumhopaasmere/tumhopaasmere, https...",PushEvent
2,(https://avatars.githubusercontent.com/u/14495...,2018-01-01T15:00:00Z,7045118892,None,"(closed, None, None, None, None, None, None, N...",True,"(62063441, daa84/neovim-gtk, https://api.githu...",PullRequestEvent
3,(https://avatars.githubusercontent.com/u/22536...,2018-01-01T15:00:00Z,7045118894,None,"(None, e61c594bc4eece3258944d5a89333e072ea0f9b...",True,"(102974522, thautwarm/EBNFParser, https://api....",PushEvent
4,(https://avatars.githubusercontent.com/u/18603...,2018-01-01T15:00:00Z,7045118895,None,"(started, None, None, None, None, None, None, ...",True,"(94912555, IVMachiavelli/awesome-threat-intell...",WatchEvent
...,...,...,...,...,...,...,...,...
495,(https://avatars.githubusercontent.com/u/37296...,2018-01-01T15:00:34Z,7045119833,(https://avatars.githubusercontent.com/u/75921...,"(created, None, ((Row(href='https://github.com...",True,"(104899694, ViacomInc/data-point, https://api....",PullRequestReviewCommentEvent
496,(https://avatars.githubusercontent.com/u/42092...,2018-01-01T15:00:35Z,7045119828,None,"(None, aeb3722b6922be28a99656a9ed965cc64c1143d...",True,"(90793418, sharkdp/fd, https://api.github.com/...",PushEvent
497,(https://avatars.githubusercontent.com/u/47457...,2018-01-01T15:00:35Z,7045119831,(https://avatars.githubusercontent.com/u/28109...,"(started, None, None, None, None, None, None, ...",True,"(20572674, GoogleCloudPlatform/appengine-guest...",WatchEvent
498,(https://avatars.githubusercontent.com/u/66829...,2018-01-01T15:00:35Z,7045119835,None,"(None, 90885d5c6c8aed4437cbc1da6f2650bb2fa3cb6...",True,"(97850341, risentveber/speedtracker, https://a...",PushEvent


This is a large file and the previous query output 500 JSON objects. To look closer, let us start looking at just the first object with a number predicate.

In [11]:
%%jsoniq
0

There was an error on line 1 in file:/Users/ghislain/Code/rumble/:

{ "kind": "object", "baseType": "js:object", "treeDepth": 3, "closed": true, "content": [ { "name": "atomic0d08af5d-10bb-4a73-af84-c6aac917a830", "type": "{ "kind": "object", "baseType": "js:object", "treeDepth": 3, "closed": true, "content": [ { "name": "actor", "type": "{ "kind": "object", "baseType": "js:object", "treeDepth": 3, "closed": true, "content": [ { "name": "avatar_url", "type": "xs:string" }, { "name": "display_login", "type": "xs:string" }, { "name": "gravatar_id", "type": "xs:string" }, { "name": "id", "type": "xs:integer" }, { "name": "login", "type": "xs:string" }, { "name": "url", "type": "xs:string" } ] }" }, { "name": "created_at", "type": "xs:string" }, { "name": "id", "type": "xs:string" }, { "name": "org", "type": "{ "kind": "object", "baseType": "js:object", "treeDepth": 3, "closed": true, "content": [ { "name": "avatar_url", "type": "xs:string" }, { "name": "gravatar_id", "type": "xs:string" }

We can see that there are nested objects and arrays. This is perfect for JSONiq. Let us now figure out all the keys used in this dataset with the keys() function.

In [116]:
%%jsoniq
keys(json-lines("http://www.rumbledb.org/samples/git-archive-small.json"))

The query output 8 items, which is too many to display. Displaying the first 5 items:
"payload"
"org"
"public"
"repo"
"type"


Let us look closer at the key called "type". What values does it take? We can use dot-based navigation to navigate down to these values. This will work nicely on the entire dataset.

In [117]:
%%jsoniq
json-lines("http://www.rumbledb.org/samples/git-archive-small.json").type

The query output 500 items, which is too many to display. Displaying the first 5 items:
"PushEvent"
"PushEvent"
"PullRequestEvent"
"PushEvent"
"WatchEvent"


It looks like there are a lot of duplicates in there. Let us use distinct-values() to figure out all unique values.

In [118]:
%%jsoniq
distinct-values(json-lines("http://www.rumbledb.org/samples/git-archive-small.json").type)

The query output 13 items, which is too many to display. Displaying the first 5 items:
"CommitCommentEvent"
"GollumEvent"
"CreateEvent"
"WatchEvent"
"IssuesEvent"


So we see that for the key "type", all values are strings and there are only... how many, by the way? Let us use count().

In [119]:
%%jsoniq
count(distinct-values(json-lines("http://www.rumbledb.org/samples/git-archive-small.json").type))

13


So there are 13. Note that count() works just as well on the entire dataset, to know how many objects there are.

In [120]:
%%jsoniq
count(json-lines("http://www.rumbledb.org/samples/git-archive-small.json"))

500


Let us know look at nested objects. It seems the key "actor" has these, so let us now use the dot object lookup to find all these values.

In [121]:
%%jsoniq
json-lines("http://www.rumbledb.org/samples/git-archive-small.json").actor

The query output 500 items, which is too many to display. Displaying the first 5 items:
{
  "id": 20090775,
  "login": "lainrose",
  "display_login": "lainrose",
  "gravatar_id": "",
  "url": "https://api.github.com/users/lainrose",
  "avatar_url": "https://avatars.githubusercontent.com/u/20090775?"
}
{
  "id": 17426563,
  "login": "tumhopaasmere",
  "display_login": "tumhopaasmere",
  "gravatar_id": "",
  "url": "https://api.github.com/users/tumhopaasmere",
  "avatar_url": "https://avatars.githubusercontent.com/u/17426563?"
}
{
  "id": 1449578,
  "login": "daa84",
  "display_login": "daa84",
  "gravatar_id": "",
  "url": "https://api.github.com/users/daa84",
  "avatar_url": "https://avatars.githubusercontent.com/u/1449578?"
}
{
  "id": 22536460,
  "login": "thautwarm",
  "display_login": "thautwarm",
  "gravatar_id": "",
  "url": "https://api.github.com/users/thautwarm",
  "avatar_url": "https://avatars.githubusercontent.com/u/22536460?"
}
{
  "id": 18603467,
  "login": "markstachowsk

We can chain dot object lookups to navigate further down, for example to logins. Let us figure out how many distinct logins there are.

In [122]:
%%jsoniq
count(distinct-values(json-lines("http://www.rumbledb.org/samples/git-archive-small.json").actor.login))

374


The id field inside the actor object seems to be an integer. What is the highest value? The max() function also works at large scales, just like count() and also min(), avg() and sum().

In [123]:
%%jsoniq
max(json-lines("http://www.rumbledb.org/samples/git-archive-small.json").actor.id)

35003609


Alright, let us know look for nested arrays. There does not seem to have any inside the actor object, so let us try the key "payload". Let us just look at the first one.

In [124]:
%%jsoniq
json-lines("http://www.rumbledb.org/samples/git-archive-small.json")[1].payload

{
  "push_id": 2226161589,
  "size": 1,
  "distinct_size": 1,
  "ref": "refs/heads/master",
  "head": "27a01fbdbec8e26daa40fc8faa052dd0be23836a",
  "before": "d6fce97b8de28a31d021c9a9f7ac939baa14d208",
  "commits": [
    {
      "sha": "27a01fbdbec8e26daa40fc8faa052dd0be23836a",
      "author": {
        "name": "lainrose",
        "email": "fb4676bf30682e2ece361fd363a69ad11779c42e@Naver.com"
      },
      "message": "Update Study Contents",
      "distinct": true,
      "url": "https://api.github.com/repos/lainrose/Python-Grammar/commits/27a01fbdbec8e26daa40fc8faa052dd0be23836a"
    }
  ]
}


Here we see that there is a nested array associated with key "commits".

In [125]:
%%jsoniq
json-lines("http://www.rumbledb.org/samples/git-archive-small.json")[1].payload.commits

[
  {
    "sha": "27a01fbdbec8e26daa40fc8faa052dd0be23836a",
    "author": {
      "name": "lainrose",
      "email": "fb4676bf30682e2ece361fd363a69ad11779c42e@Naver.com"
    },
    "message": "Update Study Contents",
    "distinct": true,
    "url": "https://api.github.com/repos/lainrose/Python-Grammar/commits/27a01fbdbec8e26daa40fc8faa052dd0be23836a"
  }
]


In this case, there is only one object in this array. Is there, by any chance, any one of these arrays that has more than one commit? For this, we can use a Boolean predicate. Let us evaluate the predicate

size($$) gt 1

which uses the size function and the gt (greater than) comparison and where $$ is the current array being tested.

In [126]:
%%jsoniq
json-lines("http://www.rumbledb.org/samples/git-archive-small.json").payload.commits[size($$) gt 1]

The query output 30 items, which is too many to display. Displaying the first 5 items:
[
  {
    "sha": "95e600df9a5a669f53dc7de28147814678d12e81",
    "author": {
      "name": "Phil Gengler",
      "email": "e888d2bd6f13f82caa51a37c03d034c76f661ba3@pgengler.net"
    },
    "message": "Get days/tasks via JSONAPI",
    "distinct": true,
    "url": "https://api.github.com/repos/pgengler/todolist-client/commits/95e600df9a5a669f53dc7de28147814678d12e81"
  },
  {
    "sha": "d348f84df64c5473ba6a95a108e7c0263a434add",
    "author": {
      "name": "Phil Gengler",
      "email": "e888d2bd6f13f82caa51a37c03d034c76f661ba3@pgengler.net"
    },
    "message": "Update tests",
    "distinct": true,
    "url": "https://api.github.com/repos/pgengler/todolist-client/commits/d348f84df64c5473ba6a95a108e7c0263a434add"
  },
  {
    "sha": "9227c61c103ec1ee7b6dc8e126d14bc85fdf3dfd",
    "author": {
      "name": "Phil Gengler",
      "email": "e888d2bd6f13f82caa51a37c03d034c76f661ba3@pgengler.net"
    },


Let us just take the first one to have more visibility.

In [127]:
%%jsoniq
json-lines("http://www.rumbledb.org/samples/git-archive-small.json").payload.commits[size($$) gt 1][1]

[
  {
    "sha": "95e600df9a5a669f53dc7de28147814678d12e81",
    "author": {
      "name": "Phil Gengler",
      "email": "e888d2bd6f13f82caa51a37c03d034c76f661ba3@pgengler.net"
    },
    "message": "Get days/tasks via JSONAPI",
    "distinct": true,
    "url": "https://api.github.com/repos/pgengler/todolist-client/commits/95e600df9a5a669f53dc7de28147814678d12e81"
  },
  {
    "sha": "d348f84df64c5473ba6a95a108e7c0263a434add",
    "author": {
      "name": "Phil Gengler",
      "email": "e888d2bd6f13f82caa51a37c03d034c76f661ba3@pgengler.net"
    },
    "message": "Update tests",
    "distinct": true,
    "url": "https://api.github.com/repos/pgengler/todolist-client/commits/d348f84df64c5473ba6a95a108e7c0263a434add"
  },
  {
    "sha": "9227c61c103ec1ee7b6dc8e126d14bc85fdf3dfd",
    "author": {
      "name": "Phil Gengler",
      "email": "e888d2bd6f13f82caa51a37c03d034c76f661ba3@pgengler.net"
    },
    "message": "Migrate to unified List model",
    "distinct": true,
    "url": "https

We can expand it to a sequence of objects using the [] array unboxing syntax.

In [128]:
%%jsoniq
json-lines("http://www.rumbledb.org/samples/git-archive-small.json").payload.commits[size($$) gt 1][1][]

{
  "sha": "95e600df9a5a669f53dc7de28147814678d12e81",
  "author": {
    "name": "Phil Gengler",
    "email": "e888d2bd6f13f82caa51a37c03d034c76f661ba3@pgengler.net"
  },
  "message": "Get days/tasks via JSONAPI",
  "distinct": true,
  "url": "https://api.github.com/repos/pgengler/todolist-client/commits/95e600df9a5a669f53dc7de28147814678d12e81"
}
{
  "sha": "d348f84df64c5473ba6a95a108e7c0263a434add",
  "author": {
    "name": "Phil Gengler",
    "email": "e888d2bd6f13f82caa51a37c03d034c76f661ba3@pgengler.net"
  },
  "message": "Update tests",
  "distinct": true,
  "url": "https://api.github.com/repos/pgengler/todolist-client/commits/d348f84df64c5473ba6a95a108e7c0263a434add"
}
{
  "sha": "9227c61c103ec1ee7b6dc8e126d14bc85fdf3dfd",
  "author": {
    "name": "Phil Gengler",
    "email": "e888d2bd6f13f82caa51a37c03d034c76f661ba3@pgengler.net"
  },
  "message": "Migrate to unified List model",
  "distinct": true,
  "url": "https://api.github.com/repos/pgengler/todolist-client/commits/9227c

We can also lookup a specific position, say, the second object, with the [[ ]] array lookup syntax.

In [129]:
%%jsoniq
json-lines("http://www.rumbledb.org/samples/git-archive-small.json").payload.commits[size($$) gt 1][1][[2]]

{
  "sha": "d348f84df64c5473ba6a95a108e7c0263a434add",
  "author": {
    "name": "Phil Gengler",
    "email": "e888d2bd6f13f82caa51a37c03d034c76f661ba3@pgengler.net"
  },
  "message": "Update tests",
  "distinct": true,
  "url": "https://api.github.com/repos/pgengler/todolist-client/commits/d348f84df64c5473ba6a95a108e7c0263a434add"
}


And now, please hold for something awesome. We can unbox all arrays of the entire collection at the same time by just using the [] syntax on the entire dataset.

In [130]:
%%jsoniq
json-lines("http://www.rumbledb.org/samples/git-archive-small.json").payload.commits[]

The query output 422 items, which is too many to display. Displaying the first 5 items:
{
  "sha": "27a01fbdbec8e26daa40fc8faa052dd0be23836a",
  "author": {
    "name": "lainrose",
    "email": "fb4676bf30682e2ece361fd363a69ad11779c42e@Naver.com"
  },
  "message": "Update Study Contents",
  "distinct": true,
  "url": "https://api.github.com/repos/lainrose/Python-Grammar/commits/27a01fbdbec8e26daa40fc8faa052dd0be23836a"
}
{
  "sha": "45b2f857540d7d4286d1abef204aef167190be0f",
  "author": {
    "name": "tumhopaasmere",
    "email": "bcc6c59276ad7bbcd0b972dd58baaef7cccc22d4@mailinator.com"
  },
  "message": "GIT CloneShare Commit",
  "distinct": true,
  "url": "https://api.github.com/repos/tumhopaasmere/tumhopaasmere/commits/45b2f857540d7d4286d1abef204aef167190be0f"
}
{
  "sha": "ea291a9baea441ea815e822bba5e8c9f330542f7",
  "author": {
    "name": "thautwarm",
    "email": "820a7b45b87f3c40f5e1c273015816c9c19a8401@outlook.com"
  },
  "message": "API overview and example",
  "distinct": tr

These are objects. It is all too tempting to navigate further down with more dot object-lookup syntax. All at the same time, obviously. Let us figure out how many unique emails there are in all commits of all events.

In [131]:
%%jsoniq
count(distinct-values(json-lines("http://www.rumbledb.org/samples/git-archive-small.json").payload.commits[].author.email))

256


Now, how many unique emails are there in first commits?

In [132]:
%%jsoniq
count(distinct-values(json-lines("http://www.rumbledb.org/samples/git-archive-small.json").payload.commits[[1]].author.email))

233


You have now learned how to navigate large JSON datasets with the dot object lookup syntax, the [] array unboxing syntax, the [[ ]] array lookup syntax, number predicates, and Boolean predicates.

All of these work nicely on very large sequences, and you can chain them arbitrarily. In fact, this will all happen in parallel on the cores of your machine or even on a large cluster.

You also saw how to aggregate large sequences of values with min, max, count, avg and sum.

Finally, you saw how to eliminate duplicates with distinct-values.

# Variables

Some of the queries seen previously involve several chained lookups and function calls. It can become complex

In [133]:
%%jsoniq
count(distinct-values(json-lines("http://www.rumbledb.org/samples/git-archive-small.json").actor.login))

374


It is then a natural thing to use variables to store intermediate results. This can be achieved with a series of let clauses. The final result is then put in a return clause.

In [134]:
%%jsoniq
let $path := "http://www.rumbledb.org/samples/git-archive-small.json"
let $events := json-lines($path)
let $actors := $events.actor
let $logins := $actors.login
let $distinct-logins := distinct-values($logins)
return count($distinct-logins)

374


Note that types are not needed, however they exist! It is possible to add a static type to each variable.
Since values can be sequences, you can add suffixes for cardinality: * for a sequence of arbitrary length, ? for zero or one item, + for one or more items.

In [135]:
%%jsoniq
let $path as string := "http://www.rumbledb.org/samples/git-archive-small.json"
let $events as object* := json-lines($path)
let $actors as object* := $events.actor
let $logins as string* := $actors.login
let $distinct-logins as string* := distinct-values($logins)
let $count as integer := count($distinct-logins)
return $count

374


As you can see, variables can be used to store single items, as well as enormous sequences. RumbleDB will automatically select the best way to evaluate your query.

Note that it is possible to reuse variable names. However, these are not assignments: these are bindings. Reusing a variable name hides the previous binding.

In [136]:
%%jsoniq
let $v as string := "http://www.rumbledb.org/samples/git-archive-small.json"
let $v as object* := json-lines($v)
let $v as object* := $v.actor
let $v as string* := $v.login
let $v as string* := distinct-values($v)
let $v as integer := count($v)
return $v

374


## Iteration

It is possible to iterate on the elements in a sequence, like so:

In [137]:
%%jsoniq
for $i in 1 to 10
return $i * 2

The query output 10 items, which is too many to display. Displaying the first 5 items:
2
4
6
8
10


The sequence to iterator on can itself come from a dataset, such as the one we were using previously:

In [138]:
%%jsoniq
for $event in json-lines("http://www.rumbledb.org/samples/git-archive-small.json")
return size($event.payload.commits)

    atomic0d08af5d-10bb-4a73-af84-c6aac917a830
0                                            1
1                                            1
2                                            1
3                                            0
4                                            4
..                                         ...
295                                          1
296                                          1
297                                          1
298                                          1
299                                          1

[300 rows x 1 columns]


For clauses can be mixed with let clauses:

In [139]:
%%jsoniq
let $path := "http://www.rumbledb.org/samples/git-archive-small.json"
for $event in json-lines($path)
let $commits := $event.payload.commits
return size($commits)

    atomic0d08af5d-10bb-4a73-af84-c6aac917a830
0                                            1
1                                            1
2                                            1
3                                            0
4                                            4
..                                         ...
295                                          1
296                                          1
297                                          1
298                                          1
299                                          1

[300 rows x 1 columns]


And the results can also be nested in a more complex query: for example, let us compute the max of all these array sizes.

In [140]:
%%jsoniq
max(
  let $path := "http://www.rumbledb.org/samples/git-archive-small.json"
  for $event in subsequence(json-lines($path),1,1)
  return 1
)

1


A third kind of clause is the where clause: it allows you to filter events. Let us only keep those with more than 10 commits, and count them.

In [141]:
%%jsoniq
count(
  let $path := "http://www.rumbledb.org/samples/git-archive-small.json"
  for $event in json-lines($path)
  let $commits := $event.payload.commits
  where size($commits) gt 10
  return $event
)

6


## Simple calculations

Let us now look closer arithmetics, comparison and logic expressions. They are particularly useful in a where clause or in a Boolean predicate, however these expressions can be used just about anywhere as this is a functional language.

### Arithmetics

JSONiq works like a calculator and can do arithmetics with the four basic operations.

In [142]:
%%jsoniq
 (38 + 2) div 2 + 11 * 2


42


(mind the division operator which is the "div" keyword. The slash operator has different semantics).

Like JSON, JSONiq works with decimals and doubles:

In [143]:
%%jsoniq
 6.022e23 * 42

2.52924e+25


JSONiq also support modulos, integer division, and has a rich function library (trigonometry, logarithms, exponential, powers, etc).

## Comparison

Values (numbers, strings, dates, etc) can be compared with the binary operators eq, ne, gt, ge, lt and le.
Let us change the comparison used in the where clause with other kinds.

In [144]:
%%jsoniq
count(
  let $path := "http://www.rumbledb.org/samples/git-archive-small.json"
  for $event in json-lines($path)
  let $commits := $event.payload.commits
  where size($commits) gt 10
  return $event
)

6


In [145]:
%%jsoniq
count(
  let $path := "http://www.rumbledb.org/samples/git-archive-small.json"
  for $event in json-lines($path)
  let $commits := $event.payload.commits
  where size($commits) eq 10
  return $event
)

0


In [146]:
%%jsoniq
count(
  let $path := "http://www.rumbledb.org/samples/git-archive-small.json"
  for $event in json-lines($path)
  let $commits := $event.payload.commits
  where size($commits) ne 10
  return $event
)

300


In [147]:
%%jsoniq
count(
  let $path := "http://www.rumbledb.org/samples/git-archive-small.json"
  for $event in json-lines($path)
  let $commits := $event.payload.commits
  where size($commits) le 10
  return $event
)

294


Why not = or < or >=? This is because these are more powerful. In fact, they implicitly perform an existential quantification over the operands.

In [148]:
%%jsoniq
1 to 10 = 5

true


In [149]:
%%jsoniq
1 to 10 > 11 to 20

false


### Logical operations

JSONiq supports Boolean logic.

In [150]:
%%jsoniq
true and false

false


In [151]:
%%jsoniq
(true or false) and (false or true)

true


The unary not is also available:

In [152]:
%%jsoniq
not true

false


Note that JSONiq, unlike SQL, does two-valued logic. Nulls are automatically converted to false.

In [153]:
%%jsoniq
null and true

false


Some non-Booleans can also get converted. For example, non-empty strings are converted to true and empty strings to false.

In [154]:
%%jsoniq
not ""

true


In [155]:
%%jsoniq
not "non empty"

false


Zero is converted to false, non-zero numbers to true.

In [156]:
%%jsoniq
not 0

true


In [157]:
%%jsoniq
not 1e10

false


### Strings

JSONiq is capable of manipulating strings as well, using functions:


In [158]:
%%jsoniq
concat("Hello ", "Captain ", "Kirk")

"Hello Captain Kirk"


In [159]:
%%jsoniq
substring("Mister Spock", 8, 5)

"Spock"


JSONiq comes up with a rich string function library out of the box, inherited from its base language. These functions are listed [here](https://www.w3.org/TR/xpath-functions-30/) (actually, you will find many more for numbers, dates, etc).



### Sequences

Until now, we have only been working with single values (an object, an array, a number, a string, a boolean). JSONiq supports sequences of values. You can build a sequence using commas:


In [160]:
%%jsoniq
 (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

The query output 10 items, which is too many to display. Displaying the first 5 items:
1
2
3
4
5


In [161]:
%%jsoniq
1, true, 4.2e1, "Life"

1
true
42
"Life"


The "to" operator is very convenient, too:

In [162]:
%%jsoniq
 (1 to 100)

The query output 100 items, which is too many to display. Displaying the first 5 items:
1
2
3
4
5


Some functions even work on sequences:

In [163]:
%%jsoniq
sum(1 to 100)

5050


In [164]:
%%jsoniq
string-join(("These", "are", "some", "words"), "-")

"These-are-some-words"


In [165]:
%%jsoniq
count(10 to 20)

11


In [166]:
%%jsoniq
avg(1 to 100)

50.5


Unlike arrays, sequences are flat. The sequence (3) is identical to the integer 3, and (1, (2, 3)) is identical to (1, 2, 3).

and even filter out some values:

In [167]:
%%jsoniq
let $sequence := 1 to 10
for $value in $sequence
let $square := $value * 2
where $square < 10
return $square

2
4
6
8


Note that you can only iterate over sequences, not arrays. To iterate over an array, you can obtain the sequence of its values with the [] operator, like so:


In [168]:
%%jsoniq
[1, 2, 3][]

1
2
3


### Conditions

You can make the output depend on a condition with an if-then-else construct:

In [169]:
%%jsoniq
for $x in 1 to 10
return if ($x < 5) then $x
                   else -$x

The query output 10 items, which is too many to display. Displaying the first 5 items:
1
2
3
4
-5


Note that the else clause is required - however, it can be the empty sequence () which is often when you need if only the then clause is relevant to you.

### Composability of Expressions

Now that you know of a couple of elementary JSONiq expressions, you can combine them in more elaborate expressions. For example, you can put any sequence of values in an array:

In [170]:
%%jsoniq
[ 1 to 10 ]

[
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10
]


Or you can dynamically compute the value of object pairs (or their key):

In [171]:
%%jsoniq
{
      "Greeting" : (let $d := "Mister Spock"
                    return concat("Hello, ", $d)),
      "Farewell" : string-join(("Live", "long", "and", "prosper"),
                               " ")
}

{
  "Greeting": "Hello, Mister Spock",
  "Farewell": "Live long and prosper"
}


You can dynamically generate object singletons (with a single pair):


In [172]:
%%jsoniq
{ concat("Integer ", 2) : 2 * 2 }

{
  "Integer 2": 4
}


and then merge lots of them into a new object with the {| |} notation:

In [173]:
%%jsoniq
{|
    for $i in 1 to 10
    return { concat("Square of ", $i) : $i * $i }
|}

{
  "Square of 1": 1,
  "Square of 2": 4,
  "Square of 3": 9,
  "Square of 4": 16,
  "Square of 5": 25,
  "Square of 6": 36,
  "Square of 7": 49,
  "Square of 8": 64,
  "Square of 9": 81,
  "Square of 10": 100
}


## JSON Navigation

Up to now, you have learnt how to compose expressions so as to do some computations and to build objects and arrays. It also works the other way round: if you have some JSON data, you can access it and navigate.
All you need to know is: JSONiq views
an array as an ordered list of values,
an object as a set of name/value pairs


### Objects

You can use the dot operator to retrieve the value associated with a key. Quotes are optional, except if the key has special characters such as spaces. It will return the value associated thereto:

In [174]:
%%jsoniq
let $person := {
    "first name" : "Sarah",
    "age" : 13,
    "gender" : "female",
    "friends" : [ "Jim", "Mary", "Jennifer"]
}
return $person."first name"

"Sarah"


You can also ask for all keys in an object:

In [175]:
%%jsoniq
let $person := {
    "name" : "Sarah",
    "age" : 13,
    "gender" : "female",
    "friends" : [ "Jim", "Mary", "Jennifer"]
}
return { "keys" : [ keys($person)] }

{
  "keys": [
    "name",
    "age",
    "gender",
    "friends"
  ]
}


### Arrays

The [[]] operator retrieves the entry at the given position:

In [176]:
%%jsoniq
let $friends := [ "Jim", "Mary", "Jennifer"]
return $friends[[1+1]]

"Mary"


It is also possible to get the size of an array:

In [177]:
%%jsoniq
let $person := {
    "name" : "Sarah",
    "age" : 13,
    "gender" : "female",
    "friends" : [ "Jim", "Mary", "Jennifer"]
}
return { "how many friends" : size($person.friends) }

{
  "how many friends": 3
}


Finally, the [] operator returns all elements in an array, as a sequence:

In [178]:
%%jsoniq
let $person := {
    "name" : "Sarah",
    "age" : 13,
    "gender" : "female",
    "friends" : [ "Jim", "Mary", "Jennifer"]
}
return $person.friends[]

"Jim"
"Mary"
"Jennifer"


### Relational Algebra

Do you remember SQL's SELECT FROM WHERE statements? JSONiq inherits selection, projection and join capability from XQuery, too.

In [179]:
%%jsoniq
let $stores :=
[
    { "store number" : 1, "state" : "MA" },
    { "store number" : 2, "state" : "MA" },
    { "store number" : 3, "state" : "CA" },
    { "store number" : 4, "state" : "CA" }
]
let $sales := [
    { "product" : "broiler", "store number" : 1, "quantity" : 20  },
    { "product" : "toaster", "store number" : 2, "quantity" : 100 },
    { "product" : "toaster", "store number" : 2, "quantity" : 50 },
    { "product" : "toaster", "store number" : 3, "quantity" : 50 },
    { "product" : "blender", "store number" : 3, "quantity" : 100 },
    { "product" : "blender", "store number" : 3, "quantity" : 150 },
    { "product" : "socks", "store number" : 1, "quantity" : 500 },
    { "product" : "socks", "store number" : 2, "quantity" : 10 },
    { "product" : "shirt", "store number" : 3, "quantity" : 10 }
]
let $join :=
    for $store in $stores[], $sale in $sales[]
    where $store."store number" = $sale."store number"
    return {
        "nb" : $store."store number",
        "state" : $store.state,
        "sold" : $sale.product
    }
return [$join]

[
  {
    "nb": 1,
    "state": "MA",
    "sold": "broiler"
  },
  {
    "nb": 1,
    "state": "MA",
    "sold": "socks"
  },
  {
    "nb": 2,
    "state": "MA",
    "sold": "toaster"
  },
  {
    "nb": 2,
    "state": "MA",
    "sold": "toaster"
  },
  {
    "nb": 2,
    "state": "MA",
    "sold": "socks"
  },
  {
    "nb": 3,
    "state": "CA",
    "sold": "toaster"
  },
  {
    "nb": 3,
    "state": "CA",
    "sold": "blender"
  },
  {
    "nb": 3,
    "state": "CA",
    "sold": "blender"
  },
  {
    "nb": 3,
    "state": "CA",
    "sold": "shirt"
  }
]


### Access datasets

RumbleDB can read input from many file systems and many file formats. If you are using our backend, you can only use json-doc() with any URI pointing to a JSON file and navigate it as you see fit. 

You can read data from your local disk, from S3, from HDFS, and also from the Web. For this tutorial, we'll read from the Web because, well, we are already on the Web.

We have put a sample at http://rumbledb.org/samples/products-small.json that contains 100,000 small objects like:



In [180]:
%%jsoniq
json-lines("http://rumbledb.org/samples/products-small.json", 10)[1]

{
  "product": "blender",
  "store-number": 20,
  "quantity": 920
}


The second parameter to json-lines, 10, indicates to RumbleDB that it should organize the data in ten partitions after downloading it, and process it in parallel. If you were reading from HDFS or S3, the parallelization of these partitions would be pushed down to the distributed file system.

JSONiq supports the relational algebra. For example, you can do a selection with a where clause, like so:

In [181]:
%%jsoniq
for $product in json-lines("http://rumbledb.org/samples/products-small.json", 10)
where $product.quantity ge 995
return $product

The query output 600 items, which is too many to display. Displaying the first 5 items:
{
  "product": "toaster",
  "store-number": 97,
  "quantity": 997
}
{
  "product": "phone",
  "store-number": 100,
  "quantity": 1000
}
{
  "product": "tv",
  "store-number": 96,
  "quantity": 996
}
{
  "product": "socks",
  "store-number": 99,
  "quantity": 999
}
{
  "product": "shirt",
  "store-number": 95,
  "quantity": 995
}


Notice that by default only the first 200 items are shown. In a typical setup, it is possible to output the result of a query to a distributed system, so it is also possible to output all the results if needed. In this case, however, as this is printed on your screen, it is more convenient not to materialize the entire sequence.

For a projection, there is project():

In [182]:
%%jsoniq
for $product in json-lines("http://rumbledb.org/samples/products-small.json", 10)
where $product.quantity ge 995
return project($product, ("store-number", "product"))

The query output 600 items, which is too many to display. Displaying the first 5 items:
{
  "store-number": 97,
  "product": "toaster"
}
{
  "store-number": 100,
  "product": "phone"
}
{
  "store-number": 96,
  "product": "tv"
}
{
  "store-number": 99,
  "product": "socks"
}
{
  "store-number": 95,
  "product": "shirt"
}


You can also page the results (like OFFSET and LIMIT in SQL) with a count clause and a where clause

In [183]:
%%jsoniq
for $product in json-lines("http://rumbledb.org/samples/products-small.json", 10)
where $product.quantity ge 995
count $c
where $c gt 10 and $c le 20
return project($product, ("store-number", "product"))

The query output 10 items, which is too many to display. Displaying the first 5 items:
{
  "store-number": 95,
  "product": "blender"
}
{
  "store-number": 98,
  "product": "tv"
}
{
  "store-number": 97,
  "product": "shirt"
}
{
  "store-number": 100,
  "product": "toaster"
}
{
  "store-number": 96,
  "product": "blender"
}


JSONiq also supports grouping with a group by clause:

In [184]:
%%jsoniq
for $product in json-lines("http://rumbledb.org/samples/products-small.json", 10)
group by $store-number := $product.store-number
return {
    "store" : $store-number,
    "count" : count($product)
}

The query output 100 items, which is too many to display. Displaying the first 5 items:
{
  "store": 1,
  "count": 1000
}
{
  "store": 2,
  "count": 1000
}
{
  "store": 3,
  "count": 1000
}
{
  "store": 4,
  "count": 1000
}
{
  "store": 5,
  "count": 1000
}


As well as ordering with an order by clause:

In [185]:
%%jsoniq
for $product in json-lines("http://rumbledb.org/samples/products-small.json", 10)
group by $store-number := $product.store-number
order by $store-number ascending
return {
    "store" : $store-number,
    "count" : count($product)
}

The query output 100 items, which is too many to display. Displaying the first 5 items:
{
  "store": 1,
  "count": 1000
}
{
  "store": 2,
  "count": 1000
}
{
  "store": 3,
  "count": 1000
}
{
  "store": 4,
  "count": 1000
}
{
  "store": 5,
  "count": 1000
}


JSONiq supports denormalized data, so you are not forced to aggregate after a grouping, you can also nest data like so:

In [186]:
%%jsoniq
for $product in json-lines("http://rumbledb.org/samples/products-small.json", 10)
group by $store-number := $product.store-number
order by $store-number ascending
return {
    "store" : $store-number,
    "products" : [ distinct-values($product.product) ]
}

The query output 100 items, which is too many to display. Displaying the first 5 items:
{
  "store": 1,
  "products": [
    "shirt",
    "toaster",
    "phone",
    "blender",
    "tv",
    "socks",
    "broiler"
  ]
}
{
  "store": 2,
  "products": [
    "shirt",
    "toaster",
    "phone",
    "blender",
    "tv",
    "socks",
    "broiler"
  ]
}
{
  "store": 3,
  "products": [
    "shirt",
    "toaster",
    "phone",
    "blender",
    "tv",
    "socks",
    "broiler"
  ]
}
{
  "store": 4,
  "products": [
    "shirt",
    "toaster",
    "phone",
    "blender",
    "tv",
    "socks",
    "broiler"
  ]
}
{
  "store": 5,
  "products": [
    "shirt",
    "toaster",
    "phone",
    "blender",
    "tv",
    "socks",
    "broiler"
  ]
}


Or

In [187]:
%%jsoniq
for $product in json-lines("http://rumbledb.org/samples/products-small.json", 10)
group by $store-number := $product.store-number
order by $store-number ascending
return {
    "store" : $store-number,
    "products" : [ project($product[position() le 10], ("product", "quantity")) ],
    "inventory" : sum($product.quantity)
}

The query output 100 items, which is too many to display. Displaying the first 5 items:
{
  "store": 1,
  "products": [
    {
      "product": "shirt",
      "quantity": 901
    },
    {
      "product": "toaster",
      "quantity": 801
    },
    {
      "product": "phone",
      "quantity": 701
    },
    {
      "product": "blender",
      "quantity": 601
    },
    {
      "product": "tv",
      "quantity": 501
    },
    {
      "product": "socks",
      "quantity": 401
    },
    {
      "product": "broiler",
      "quantity": 301
    },
    {
      "product": "shirt",
      "quantity": 201
    },
    {
      "product": "toaster",
      "quantity": 101
    },
    {
      "product": "phone",
      "quantity": 1
    }
  ],
  "inventory": 451000
}
{
  "store": 2,
  "products": [
    {
      "product": "shirt",
      "quantity": 602
    },
    {
      "product": "toaster",
      "quantity": 502
    },
    {
      "product": "phone",
      "quantity": 402
    },
    {
      "product":

That's it! You know the basics of JSONiq. Now you can also download the RumbleDB jar and run it on your own laptop. Or [on a Spark cluster, reading data from and to HDFS](https://rumble.readthedocs.io/en/latest/Run%20on%20a%20cluster/), etc.
